# Creating a frequency finder auto-encoder

## Import Libraries

In [2]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import callbacks
import tensorflow.keras as keras
from tensorflow.keras import layers
import umap
from babyplots import Babyplot
#import hdbscan
from matplotlib import pyplot
from models import *
from converters import *
from helpers import *
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [3]:
%load_ext autoreload
%autoreload 2

## Get drive path 

In [6]:
drive_path = find_drive_path()
drive_path

'/media/mat2m10/Samsung_T5'

## Load Data from drive

In [7]:
data = pd.read_pickle(f"{drive_path}/Data/Inputs/Synthetisch/Syn_tot.pkl")
data.head()

,IID,Pheno,Geno,State
0,0pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, ...",pop1sick
1,1pop1,1,"[2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, ...",pop1sick
2,2pop1,1,"[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, ...",pop1sick
3,3pop1,0,"[2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",pop1healthy
4,4pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, ...",pop1sick


In [8]:
data.tail()

,IID,Pheno,Geno,State
1995,995pop2,0,"[2, 0, 0, 0, 1, 2, 2, 2, 1, 0, 1, 0, 2, 1, 1, ...",pop2healthy
1996,996pop2,0,"[1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, ...",pop2healthy
1997,997pop2,0,"[1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 1, 2, ...",pop2healthy
1998,998pop2,0,"[2, 2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 2, 2, 0, 0, ...",pop2healthy
1999,999pop2,0,"[1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 2, 1, 1, 1, 1, ...",pop2healthy


2.5.0


In [5]:
import tensorflow as tf

In [10]:
tf.config.list_physical_devices("GPU")

[]

2021-10-01 11:34:56.481486: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-01 11:34:56.535759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-01 11:34:56.536133: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 Ti computeCapability: 7.5
coreClock: 1.485GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 178.84GiB/s
2021-10-01 11:34:56.536227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-01 11:34:56.536298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library '